In [1]:
%load_ext autoreload
%autoreload 2

import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm.auto import tqdm
from IPython.display import clear_output
from pathlib import Path

from PIL import Image

from loader import DataLoader
from registration.first_registration import Align
from models.ransacflow import register_image

DATA_DIR = "/Users/farhanoktavian/imperial/thesis/sandbox/dataset/raw"
METADATA_DIR = "/Users/farhanoktavian/imperial/thesis/sandbox/dataset/data.h5"
STACKREG_DIR = Path("/Users/farhanoktavian/imperial/thesis/sandbox/dataset/stackreg5")

run_loader = DataLoader(DATA_DIR, METADATA_DIR)

Loaded pretrained model from /Users/farhanoktavian/imperial/thesis/sandbox/rf_regist/weights/MegaDepth_Theta1_Eta001_Grad1_0.774.pth


In [4]:
sample = run_loader.samples[18]
run = sample.runs[2]
print(f"{sample.sample_id} - {run.run_id}")

HS020 - Run06


In [6]:
target_wavelength = 600

start_t = time.time()

for c in range(run.collection_count):
    print(f"Collection {c+1}/{run.collection_count}")
    imgs, img_paths = run.get_spectral_images(collection_idx=c, with_path=True)
    tgt_img = run.get_spectral_image(wavelength=target_wavelength, collection_idx=c)
    for i in tqdm(range(len(imgs))):
        img = imgs[i]
        img_path = img_paths[i]

        blank_img = np.zeros_like(img)
        temp_img = cv2.merge([img, blank_img, blank_img])

        blank_tgt_img = np.zeros_like(tgt_img)
        temp_tgt_img = cv2.merge([tgt_img, blank_tgt_img, blank_tgt_img])

        img = register_image(Image.fromarray(temp_img), Image.fromarray(temp_tgt_img))
        img = img[:, :, 0]

        save_path = STACKREG_DIR / sample.sample_id / run.run_id
        save_path.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(save_path / img_path.name), img)

end_t = time.time()
print(f"Time taken: {end_t - start_t}s")

Collection 1/4


100%|██████████| 29/29 [01:51<00:00,  3.85s/it]


Collection 2/4


100%|██████████| 29/29 [01:52<00:00,  3.87s/it]


Collection 3/4


100%|██████████| 29/29 [01:55<00:00,  3.98s/it]


Collection 4/4


100%|██████████| 29/29 [01:52<00:00,  3.87s/it]

Time taken: 453.9492838382721s
